# Lecture 10: Backpropagation and Gradient Descent
In this lecture, we will discuss the machinery behind how training neural nets works. To do this, we will review some calculus, perform backpropagation on a simple node, and build up to a fully complex neural network

as a quick expansion of what this loss function means, let's right it out ourselves and take a look

Let's first look at what the softmax function does. Softmax is defined by the equation

$F({Y'}_i) = \frac{e^{{Y'}_i}}{\sum_{j=0}^{k} e^{{Y'}_j}}$

Although it looks a little funky, it basically forces all the values to in $F({Y'})$ to sum to one and represent the probability of each ${Y'}_i$ being true.

In [3]:
import numpy as np
def softmax(X):
    output = np.empty(X.shape)
    for i, entry in enumerate(X):
        output[i] = np.exp(entry) / np.sum(np.exp(X))
    return output

In [4]:
output = softmax(np.asarray([1,2,3,4]))
output

array([0.0320586 , 0.08714432, 0.23688282, 0.64391426])

We can see above that the sum of the output is 1 and the larger each input is, the higher the probability that it's the right choice. You can see how this is quite useful when we have to choose which class a network should guess!

Next, let's look at cross entropy loss. Cross entropy is a simple way of measuring how far a guess is from the correct answer. It is defined as:

$CE(Y', Y) = -\sum_i log({Y'}_i) \times Y_i$

Where $Y'$ are the guesses made and $Y$ are the correct labels. The log is used to make the equation easier to differentiate, which we'll take a look at later.

![CE](http://ml-cheatsheet.readthedocs.io/en/latest/_images/cross_entropy.png)

In [5]:
def CE(output, labels):
    return -np.sum(labels*np.log(output))

labels = np.asarray([0,0,0,1])
CE(output, labels)

0.44018969856119544

In [8]:
import mxnet
from mxnet import gluon, nd
loss = gluon.loss.SoftmaxCrossEntropyLoss(sparse_label=True)
input = nd.array([1,2,3,4])
labels = nd.array([3])
loss(input, labels)


[0.4401897]
<NDArray 1 @cpu(0)>

We see above that our simple code matches what MXNet is doing when we call the softmax cross entropy loss. Great! Now we know whats happening behind the scenes for the loss. 

Because cross entropy measures how good a guess is (with lower outputs meaning better guesses) our goal is always to minimize the output of the loss function. Let's take a look at a basic perceptron trying to classify MNIST and see how we can minimize the loss.

In [9]:
# import standard mxnet packages
import mxnet as mx
from mxnet import nd, autograd
from mxnet import gluon
import numpy as np

# import matplotlib for plotting
import matplotlib.pyplot as plt
%matplotlib inline

# set the device we should use for computing, we'll just our cpu for now
data_ctx = mx.cpu()
model_ctx = mx.cpu()

In [10]:
# load MNIST, a very simple image classification dataset

# set the size of the training set
num_examples = 60000
# set batch size : how many images should i process at a time?
batch_size = 64
# set the number of pixels per image (32 x 32)
num_inputs = 784
# set the number of possible outputs (0 through 9)
num_outputs = 10
# define a function that scales the image pixels down between 0 and 1
def transform(data, label):
    return data.astype(np.float32)/255, label.astype(np.float32)
# load the training data
train_data = mx.gluon.data.DataLoader([d for d in mx.gluon.data.vision.MNIST(train=True, transform=transform)],
                                      batch_size, shuffle=True)
# load the test data
test_data = mx.gluon.data.DataLoader([d for d in mx.gluon.data.vision.MNIST(train=False, transform=transform)],
                              batch_size, shuffle=False)

/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/mxnet/gluon/data/vision.py:118: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  label = np.fromstring(fin.read(), dtype=np.uint8).astype(np.int32)
/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/mxnet/gluon/data/vision.py:122: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data = np.fromstring(fin.read(), dtype=np.uint8)


In [30]:
# define a minimal neural network
net = gluon.nn.Dense(num_outputs, in_units=784, use_bias=True)
# initialize the parameters of the network randomly
net.collect_params().initialize(mx.init.Normal(), ctx=model_ctx)
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

![simplenet](http://jwfromm.com/GIX513/images/simplenet.png)

Let's forumate what's happening here a little better. Since we have a simple one layer network, we can describe it entirely by the equation:

$Y' = WX$

Where $Y'$ is the 10 outputs of the network, W is the weight matrix of our neurons, with shape (784x10), and $X$ is one input image of MNIST, with shape (1x784). This is a simple matrix multiplication that represents multiply accumulating the weights of each neuron with the corresponding input pixels.

As this is a fairly simple equation, it's clear that our goal is to find the values of W (the neurons weights) that make our guesses $Y'$ as accurate as possible.

In [12]:
# define a function to see how good our model is
def evaluate_accuracy(data_iterator, net):
    # keep track of the accuracy across our dataset
    acc = mx.metric.Accuracy()
    # iterate through all the data
    for i, (data, label) in enumerate(data_iterator):
        # move the data and label to the proper device
        data = data.as_in_context(model_ctx).reshape((-1, 784))
        label = label.as_in_context(model_ctx)
        # run the data through the network
        output = net(data)
        # check what our guess is
        predictions = nd.argmax(output, axis=1)
        # compute accuracy and update our running tally
        acc.update(preds=predictions, labels=label)
    # return the accuracy
    return acc.get()[1]

# define a function to see how good our model is based on loss
def evaluate_loss(data_iterator, net):
    # keep track of the accuracy across our dataset
    lossmetric = mx.metric.Loss()
    # iterate through all the data
    for i, (data, label) in enumerate(data_iterator):
        # move the data and label to the proper device
        data = data.as_in_context(model_ctx).reshape((-1, 784))
        label = label.as_in_context(model_ctx)
        # run the data through the network
        output = net(data)
        # check what our guess is
        loss = softmax_cross_entropy(output, label)
        # compute accuracy and update our running tally
        lossmetric.update(None, loss)
    # return the accuracy
    return lossmetric.get()[1]

In [13]:
evaluate_accuracy(test_data, net)

0.1029

As expected, the accuracy of our network is currently about equal to just guessing the output. Let's brain storm on how we can improve the weight matrix of our network to make better guesses.

## Optimization Method 1: Random Search
Let's start by trying something very simple, we'll just randomly assign numbers to $W$ and keep whatever works best!

In [32]:
# keep track of the best accuracy we've seen
bestloss = np.inf
# also keep track of the best weights
bestW = None
bestb = None
# try 1000 different weights
for i in range(10):
    # generate a random weight tensor
    W = nd.random.normal(shape=net.weight.shape)
    b = nd.random.normal(shape = net.bias.shape)
    # assign those weights to our network
    net.weight.set_data(W)
    net.bias.set_data(b)
    # check the accuracy of the network
    loss = evaluate_loss(test_data, net)
    print(loss)
    # see if this is the best we've done
    if loss < bestloss:
        bestloss = loss
        bestW = W
        bestB = b
print(bestloss)

14.283894435135567
15.636196653414835
12.048128410182942
13.206123843483219
12.839609982239052
12.887183442033605
14.293710940545486
13.375741547928177
13.425769245263018
18.982629643232638
12.048128410182942


In [33]:
net.weight.set_data(bestW)
net.bias.set_data(bestB)
evaluate_accuracy(test_data, net)

0.1156

Chances are, we did a little better than guessing, but not by much. This isnt too surprising since our search is pretty much totally braindead.

## Optimization Method 2: Iterative Refinement
Rather than try to find good weights all in one step randomly, let's try starting off with random weights, then making small incremental improvements. The idea here is that we're trying to take gradual steps to minimize the loss. 

### Blindfolded Hiker Analogy
![hiker](https://i0.wp.com/www.slowbru.com/wp-content/gallery/feature-photos/P1090479c.JPG?w=1505)

Imagine you're stuck on a mountain and you can't see anything. You're goal is to get to the bottom of the mountain to find help. How might you proceed?

You're best best is to feel the area around you and take a small step in the most downhill direction. Little step by little step, you'll work your way down to a lower point. Let's try implementing something like that.

In [36]:
# keep track of the best accuracy we've seen
bestloss = np.inf
# also keep track of the best weights
bestW = None
bestB = None
# try 1000 different weights
for i in range(30):
    step_size = 0.1
    W = net.weight.data()
    B = net.bias.data()
    # generate a random weight tensor
    Wtry = W + nd.random.normal(shape=(net.weight.shape)) * step_size
    Btry = B + nd.random.normal(shape=net.bias.shape)*step_size
    # assign those weights to our network
    net.weight.set_data(Wtry)
    net.bias.set_data(Btry)
    # check the accuracy of the network
    loss = evaluate_loss(test_data, net)
    print(loss)
    # see if this is the best we've done
    if loss < bestloss:
        bestloss = loss
        bestW = Wtry
        bestB = Btry
    else:
        # if its not better, go back
        net.weight.set_data(W)
        net.bias.set_data(B)
print(bestloss)

12.059775368073547
12.063872955708504
12.047404893531871
12.079870734349408
12.071828313856944
12.06071943607698
12.074631603229285
12.080587752273487
12.06819632284897
12.058140230914713
12.037845381003924
12.0589133524021
12.095497172233825
12.114147287001197
12.133503323581417
12.132975141884002
12.144219668126238
12.176451792935913
12.163096136879103
12.172257966694376
12.12590785873648
12.117019643220301
12.115369545439552
12.118799829523615
12.135190693686251
12.15371385667314
12.167807321086727
12.196191918441752
12.165933405781143
12.16619690394417
12.037845381003924


In [37]:
net.weight.set_data(bestW)
evaluate_accuracy(test_data, net)

0.1157

We can see that as training proceeds, the loss gradually works its way down. This is a big improvement from taking random steps!

However, right now we're trying steps in random directions, this is computationally inefficient. If you remember calculus, you'll note that we can simply use derivatives to determine which way to step!

![slope](http://www.sosmath.com/calculus/diff/der00/der00_2.gif)

![derivatives](http://hyperphysics.phy-astr.gsu.edu/hbase/Math/immath/derfunc.gif)

Let's take a look at our simple network. Remember that it's computation can be expressed as

$Y' = WX$

And the loss function is defined as

$F({Y'}_i) = \frac{e^{{Y'}_i}}{\sum_{j=0}^{k} e^{{Y'}_j}}$

$Loss = CE(Y', Y) = -\sum_i log({F(Y')}_i) \times Y_i$

Our goal is to find the slope of the Loss with respect to our weights $W$, so that we can move $W$ in the appropriate direction. This goal can be expressed as

$\frac{dL}{dW}$

This might seem like a very tough thing to compute, since there are already three equations in the way, but let's break up this complicated derivative into three parts using chain rule.

$\frac{dL}{dW} = \frac{d{Y'}}{dW} \times \frac{dF}{d{Y'}} \times \frac{dL}{dF}$

You can see that the right side cancels out to the left side, neat! This way, we can compute the derivative of each function seperate, then multiply them all together to get the full network derivative.

Lets work on computing these derivatives one by one.

$\frac{d{Y'}}{dW} = \frac{dWX}{dW} = X$

This first one is pretty easy since it's just a multiplication.

The next two get a little messy, and I'm going to avoid it since it'd be a slog to work through. There are plenty of online resources if you'd like to see it derived, but it turns out we can write

$\frac{dF}{d{Y'}} \times \frac{dL}{dF} = \frac{dL}{d{Y'}} = F(Y') - Y$

Thus, the derivative of our weights is equal to

$X \times F(Y') - Y$

Let's try it out!

In [19]:
# derivative of softmax cross entropy
# X is output of dense layer
# Y is labels
def delta_cross_entropy(X, y):
    # make label one hot for easier use
    y = nd.one_hot(y, 10)
    batch_size = y.shape[0]
    grad = nd.softmax(X)
    # subtract 1 (y) where appropriate
    grad = grad - y
    # scale by batch size
    grad = grad / batch_size
    return grad

In [20]:
# compute the gradient of matrix multiplication (dense layer).
# gradin is the gradient of the output with respect to the loss
# X is the input to the layer
def delta_matrix_mul(gradin, X):
    W_updates = nd.dot(gradin.transpose(), X)
    return W_updates

Thats not so bad! Let's try using these new tricks to do training!

Now that we know how the loss relates to the weights, we know which direction to move the weights in! Specifically, we want the loss to decrease, so we move the weights in the opposite direction of the derivative. The update for one step of training is thus

$W = W + lr \times \frac{dW}{dL} = W + lr \times (X (F(Y') - Y))$

Where lr is the learning rate. Basically, the learning rate represents how big a step we should take. Just like in blind-folded hiking, it's usually safer to take small steps rather than big ones.

Now, we implement our custom gradient descent in a training loop, it'll likely look similar, but note that we dont use autograd at all. The mxnet autograd package actually stands for automatic gradients, but here we're clearly computing our own gradients.

In [62]:
learning_rate = .1

def custom_train(net, epochs=10):
    for e in range(epochs):
        # we're going to sum up the loss over the whole epoch
        cumulative_loss = 0
        # iterate through all the training data
        for i, (data, label) in enumerate(train_data):
            # make sure the data is on the right device, flatten the images
            data = data.as_in_context(model_ctx).reshape((-1,784))
            label = label.as_in_context(model_ctx)
            # update our loss for this epoch
            output = net(data)
            loss = softmax_cross_entropy(output, label)
            # keep track of loss for printing purposes
            cumulative_loss += nd.sum(loss).asscalar()
            # now lets apply our custom gradients!
            dFdL = delta_cross_entropy(output, label)
            dWdF = delta_matrix_mul(dFdL, data)
            # derivative of bias is just 1
            dBdF = nd.mean(dFdL, axis=0)
            # use our weight gradients to update the weights and bias
            W = net.weight.data()
            B = net.bias.data()
            # update the weight and bias
            new_W = W - (learning_rate * dWdF)
            new_B = B - (learning_rate * dBdF)
            net.weight.set_data(new_W)
            net.bias.set_data(new_B)
        print("Epoch %s. Loss: %s" % (e, cumulative_loss/num_examples))

In [63]:
custom_train(net)

Epoch 0. Loss: 0.2775573268731435
Epoch 1. Loss: 0.27582611340284346
Epoch 2. Loss: 0.27451042377154034
Epoch 3. Loss: 0.27331182289123535
Epoch 4. Loss: 0.2723051885843277
Epoch 5. Loss: 0.2710574910879135
Epoch 6. Loss: 0.27040566515525183
Epoch 7. Loss: 0.26928829301198326
Epoch 8. Loss: 0.26815170016686124
Epoch 9. Loss: 0.26732689545551935


In [61]:
evaluate_accuracy(test_data, net)

0.9194

That worked much better! The difference is pretty much night and day. Although the random step method should in theory get similar results, it would take an absurd amount of time to do so.

## Computing Gradients for Large Networks: Backpropagation

![simplebackprop](http://jwfromm.com/GIX513/images/simple_backprop.png)

Here's a different way of looking at what we just did. We have an input X to a dense layer, which uses it's weights to compute Y. If we rename the dense layer as $F(X)$, then just as before want to compute.

$\frac{dW}{dY}$

Where Y could be considered the loss in this case. We know how to do this of course. But what if it were a more complicated network?

![medium](http://jwfromm.com/GIX513/images/medium_backprop.png)

Now, we have two dense layers, $F1$ and $F2$. Let's take a look at how we could compute the weight updates for both!

In this case, the loss is represented by the output node $Y2$, so thats what we'll compute our derivatives with respect to. Since $W2$ is closer to $Y2$ than $W1$ is, we'll start by finding it's gradient.

$\frac{dW_2}{dY_2} = Y_1$

Well that was easy, this is basically the same equation as when we were doing a single layer. Now lets move on to finding the gradient of $W_1$.

$\frac{dW_1}{dY_2} = ....$

hmm, theres a bunch of stuff in the way, the relationship between $W_1$ and $Y_2$ isn't directly obvious. Let's start by looking at nodes close to $W.1$ though.

$\frac{dW_1}{dY_1} = X$

This looks familiar! Now that we have the gradient with respect to $Y_1$, we can try to find $\frac{dY_1}{dY_2}$.

$\frac{dY_1}{dY_2} = W_2$

This looks similar to our dense layer, but the gradient is defined by the weight instead of the input. That's because we took the derivative of an input with respect to an output instead of a weight with respect to an output. Now we can solve for $W_1$'s full gradient.

$\frac{dW_1}{dY_2} = \frac{dW_1}{dY_1} \times \frac{dY_1}{dY_2} = X \times W_2$

And thats it! Now we have a two layer gradient. But what if its even more complicated? Let's try looking at this in a different way.

![medium](http://jwfromm.com/GIX513/images/full_backprop.png)

In the image above, we try to compute the full gradient for each weight in a 3 layer network. You might notice there are now backwards arrows, that's because we're going to allow the gradients to flow from the loss back to the weights.

You can see in the bottom dense layer the weight update is simply

$\frac{dW_3}{dY_3}$

Which is quite easy to compute.

We also know how to compute $\frac{dY_2}{dY_3}$

Next, we move up a layer. To update $W_2$ we must compute:

$\frac{dW_2}{dY_3} = \frac{dY_2}{dY_3} \times \frac{dW_2}{dY_2}$

But we've already computed $\frac{dY_2}{dY_3}$! So we need only additionally compute $\frac{dW_2}{dY_2}$. Fortunately, this single gradient is also very easy to compute!

We similary find $\frac{dY_1}{dY_3} = \frac{dY_2}{dY_3} * \frac{dY_1}{dY_2}$

Finally, we move up a layer to find $\frac{dW_1}{dY_3}$. This is expressed by

$\frac{dW_1}{dY_3} = \frac{dY_2}{dY_3} \times \frac{dY_1}{dY_2} \times \frac{dW_1}{dY_1}$

This is starting to get quite large, but we just found $\frac{dY_2}{dY_3} \times \frac{dY_1}{dY_2}$, so we need only multiply that by our current simple gradient $\frac{dW_1}{dY_1}$! This is actually quite simple since we allow the gradients to accumulate as they propagate up the network.

It turns out that no matter how complex your network is, you can make a graph similar to above and as long as each function node is differentiable, you can propagate the gradient backwards up the net to find each parameter gradient. This is a surprisingly efficient operation that is all thanks to chain-rule in calculus. 

This is exactly the computation that mxnet performs when you say

loss.backward()